# Part 3 Solutions Notebook

The following libraries will be used this part.

In [1]:
from PIL import Image
import statistics
import matplotlib.pyplot as plt
import numpy as np

> **NOTE:** The source code is available at [here](https://github.com/BU-IE-582/fall19-akinelden/blob/master/Project_Part3/Part3.ipynb) 